# Creación de cargadores de datos

En esta sección se van a crear cargadores de datos de Pythorch similares a los de las seccion_02.

Anteriormente se utilizó una técnica de ventana deslizante para generar fragmentos de texto de tamaño uniforme los cuales  luego  se  agruparon  en  lotes  para  un  entrenamiento  del  modelo  más  eficiente.
Cada  fragmento  funcionó  como  una  instancia  de  entrenamiento  individual.

Sin embargo, en esta sección, se va a trabajar  con  un  conjunto  de  datos  de  spam  que  contiene  mensajes  de  texto  de  distinta  longitud.  Para  agrupar  estos  mensajes, se tienen dos opciones:

- Truncar todos los mensajes a la longitud del mensaje más corto del conjunto de datos o lote.
- Rellena todos los mensajes hasta la longitud del mensaje más largo del conjunto de datos o lote.

La  opción  1  es  computacionalmente  más  económica,  pero  puede  resultar  en  una  pérdida  significativa  de  información  si  los  mensajes  más  cortos  son  mucho  más  pequeños  que  los  mensajes  promedio  o  más  largos,  lo  que  podría  reducir  el  rendimiento  del  modelo. 

Para  implementar  la  opción  2,  donde  todos  los  mensajes  se  rellenan  hasta  la  longitud  del  mensaje  más  largo  del  conjunto  de  datos,  se añaden  tokens  de  relleno  a  todos  los  mensajes  más  cortos.  Para  ello,  se usa  "<|endoftext|>"  como  token  de  relleno.

Sin  embargo,  en  lugar  de  agregar  la  cadena  "<|endoftext|>"  a  cada  uno  de  los  mensajes  de  texto  directamente, se puede  agregar  el  ID  de  token  correspondiente  a  "<|endoftext|>"  a  los  mensajes  de  texto  codificados.

![Texto alternativo](./imgs/6.5.png)

Suponer  que  50256  es  el  ID  del  token  de  relleno  "<|endoftext|>".
Se puede verificar  que  este  sea  el  ID  de  token  correcto  codificando  "  <|endoftext|>"  usando  el  tokenizador  GPT2  del  paquete  tiktoken.

In [1]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


Primero es necesario implementar  un  conjunto  de  datos  de  PyTorch,  para especificar  cómo  se  cargan  y  procesan  los  datos,  antes  de  que  se puedan  instanciar  los  cargadores  de  datos.

Para  ello,  se define  la  clase  SpamDataset ,  que  implementa  los  conceptos  ilustrados  en  la  figura anterior.  Esta  clase  SpamDataset  gestiona  varias  tareas  clave:  identifica  la  secuencia  más  larga  del  conjunto  de  datos  de  entrenamiento,  codifica  los  mensajes  de  texto  y  garantiza  que  todas  las  demás  secuencias  se  rellenen  con  un  token  de  relleno  para  que  coincida  con  la  longitud  de  la  secuencia  más  larga.secuencia.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset

df_train = pd.read_csv("./CSV/train.csv")
encoded_text = [tokenizer.encode(text) for text in df_train["Text"]]

maxq = 0
for text in encoded_text:
    len_1 = len(text)
    if len_1 > max:
        maxq = len_1

encoded_text = [text + ["50256"] * (maxq - len(text)) for text in encoded_text]